In [78]:
import numpy as np
import nltk
import pymorphy2 as p
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from collections import Counter

In [226]:
morph = p.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

titles = ['Потребление сладкого в России побило рекорд',
                    'ИТ-компании и мировые лидеры решили бороться с террористическим контентом',
                    'Эксперты спрогнозировали отзыв лицензий у 46 российских банков']
articles = [open('data1.txt', encoding='utf8').read(), 
            open('data2.txt', encoding='utf8').read(), 
            open('data3.txt', encoding='utf8').read()]

raw_articles = [] # list of raw articles
flat_text = [] # list of flat texts for countig word frequences
word_freq = [] # list of dicts with word as a key and frequence as value
sent_rating = [] # list of numpy arrays where each row contais [sent_#, sent_rating]
avg_sent_rating =[] # list on numpy arrays where each row contais [sent_#, avg_sent_rating]
best_sent_idx = [] # list of best sentences indices
best_avg_sent_idx = [] # list of best (avg) sentences indices

for i in range(3):
    articles[i] = sent_tokenize(articles[i]) # split by sentences
    raw_articles.append(np.copy(articles[i]))
    f_t = []
    for j in range(len(articles[i])):
        articles[i][j] = tokenizer.tokenize(articles[i][j]) # split by words
        normalized_sent = []
        for k in range(len(articles[i][j])):
            word = articles[i][j][k] # get every word
            normalized_word = morph.parse(word)[0].normal_form # normal form of word
            if normalized_word not in stopwords.words('russian'): # if word is not a stop word
                normalized_sent.append(normalized_word)
        articles[i][j] = normalized_sent # replace origin sentences by sentences with normalized words and without stop words
        sent = ' '.join(articles[i][j]) # join sentences
        f_t.append(sent)
    f_t = ' '.join(f_t) # join to text
    flat_text.append(f_t) # flat text to count frequences
    word_freq.append(Counter(flat_text[i].split(' '))) # counting frequences
    
    sent_rating.append(np.zeros((len(articles[i]), 2)))
    sent_rating[i][:, 0] = np.arange(sent_rating[i].shape[0]) # sentences rating has described structure (rating is 0)
    avg_sent_rating.append(np.zeros((len(articles[i]), 2))) # for avg_sent_rating too
    avg_sent_rating[i][:, 0] = np.arange(sent_rating[i].shape[0])
    
    for j in range(len(articles[i])):
        for k in range(len(articles[i][j])):
            word = articles[i][j][k]
            freq = word_freq[i][word]
            sent_rating[i][j][1] += freq # add each word freq
        avg_sent_rating[i][j][1] = sent_rating[i][j][1] / len(articles[i][j])
        
    sent_rating[i] = sent_rating[i][sent_rating[i][:,1].argsort()[::-1][:4]] # get the best 4 sentences
    avg_sent_rating[i] = avg_sent_rating[i][avg_sent_rating[i][:,1].argsort()[::-1][:4]] # get the best 4 sentences
    
    best_sent_idx.append(sent_rating[i][:,0])
    best_avg_sent_idx.append(avg_sent_rating[i][:,0])
    print("="*80)
    print("Article {} has title \"{}\"".format(i+1, titles[i]))
    print("Annotation with sum is:")
    for b in range(4):
        print(raw_articles[i][int(best_sent_idx[i][b])])
    print("-"*40)
    print("Annotation with avg is:")    
    for b in range(4):
        print(raw_articles[i][int(best_avg_sent_idx[i][b])])
    print("="*80 + '\n')        

Article 1 has title "Потребление сладкого в России побило рекорд"
Annotation with sum is:
Продажи кондитерских изделий, по данным «Асконда», увеличились в 2018 году на 3% в натуральном выражении — объем рынка составил 3,7 млн т. Рост продаж кондитерских изделий в денежном выражении зафиксировал и Росстат.
В первом квартале 2019 года, по данным Ipsos Comcon, наблюдается небольшой рост доли потребителей сладкого — особенно заметен рост потребления более доступных кондитерских изделий, плиток шоколада и развесных конфет, а также рост частоты потребления плиток и конфет в коробках.
В 2018 году, по данным «Асконда», российские кондитеры выпустили 3,9 млн т кондитерских изделий — на 3,7% больше, чем в предыдущем году.
По данным Nielsen, в 2018 году продажи алкогольной продукции в России показали рост на 4,6% в денежном выражении по сравнению с предыдущим годом.
----------------------------------------
Annotation with avg is:
Продажи кондитерских изделий, по данным «Асконда», увеличились в 20

# Вывод

Оба метода автоматической аннотации отработали достаточно хорошо. Практически все предложения из построенной аннотации отражают суть статьи. Исключения составили следующие предложения:

Для статьи 1:

По данным Nielsen, в 2018 году продажи алкогольной продукции в России показали рост на 4,6% в денежном выражении по сравнению с предыдущим годом. (метод суммирования)

Для статьи 2:

Как говорится в заявлении компании, впредь пользователи, нарушившие регламент соцсети, не смогут вести прямые трансляции. (метод усреднения)

Для статьи 3:

«Что касается региональных банков, — возражает Рудых. (метод усреднения)